In [3]:
import numpy as np
from typing import Sequence,Iterable, Iterator, Tuple, TypeVar, Dict, Callable,List
from rl.markov_decision_process import Policy
import math
from rl.distribution import (Bernoulli, Constant, Categorical, Choose,
                             Distribution, FiniteDistribution)

In [5]:
S = TypeVar('S')
A = TypeVar('A')
def LSTD(feature_func: Callable[[S],Sequence[float]],     # feature functions
         simulator: Callable[[S,A],Tuple[S,float]],
         p:Policy[S,A],
         d: int,
         gamma: float,
         epsilon: float,
         state_distribution: Distribution[S],
         tolerance: float = 1e-6,
         nstop: int = None
         )->Iterator[Sequence[float]]:
    """
    LSTD algorithm.
    feature_func:S->R^d. feature_func(terminal) = 0
    simulator: Take input state and action, output next state and reward
    d: dimension of features
    p: The fixed policy
    epsilon: small number for initialization

    return: Iterator of weights R^d
    """

    A_inverse = 1/epsilon*np.eye(d)
    b = np.zeros(d)

    max_steps = round(math.log(tolerance) / math.log(gamma)) if gamma < 1 else nstop

    while True:
        state = state_distribution.sample()
        x = feature_func(state)

        step_count = 0
        while step_count < max_steps:
            step_count += 1
            action = p.act(state).sample()
            next_state,reward = simulator(state,action)
            xp = feature_func(next_state)

            # update A^(-1), b, weight for each time step. Use Shermannm-Morrison incremental inverse
            v = np.dot(np.transpose(A_inverse),x-gamma*xp)
            A_inverse -= np.outer(np.dot(A_inverse,x),v)/(1+np.dot(v,x))
            b += reward*x
            weight = np.dot(A_inverse,b)

            state = next_state
            x = xp

        yield weight